In [4]:
# Setup
!pip install --quiet --upgrade pip

# Install the dependencies.
!pip install --quiet -r gdal_req.txt

# Restart the runtime by ending the process.
exit()

In [1]:
import numpy as np
import rasterio
from google.cloud import storage
import os
import matplotlib.pyplot as plt
from utils.constants import  BUCKET, IMG_SOURCE_PREFIX, HIST_DEST_PREFIX, NUM_BANDS, HIST_BINS_LIST
from serving_hist.hist_data import list_blobs_with_prefix, load_tiff_from_gcs_mem, download_and_process_tiff,load_tiff_from_gcs_temp,recombine_image
import io
import google.auth
from rasterio.io import MemoryFile
from osgeo import gdal
import time
import logging

In [2]:
# SETUP
credentials, _ = google.auth.default()

bucket_name = BUCKET

directory_prefix = IMG_SOURCE_PREFIX
output_prefix = HIST_DEST_PREFIX

logging.basicConfig(filename="hist.log",level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [4]:

image_name = r"images/Canyon_2017_5-6_100"
# image_name = r"images/Story_2018_9-10_100_.tif"
blobs = list_blobs_with_prefix(bucket_name, image_name)


# Usage
start_time = time.time()

recombine_image_hist = recombine_image(image_name)

In [5]:
recombine_image_hist

array([], dtype=float64)

In [21]:
start_time = time.time()
storage_client = storage.Client()
blob = storage_client.bucket(BUCKET).blob(blob_name)
with blob.open("rb") as f:
     with rasterio.open(f) as src:
            num_bands = src.count
            _, _, hist = process_band(src, 1, bins)

end_time = time.time()
execution_time = end_time - start_time
print(f"Read and process one img band: {execution_time:.4f} seconds")  

Read and process one img band: 11.5494 seconds
